# Uncertainty-Aware CheXpert Diagnosis - Exploration

This notebook demonstrates the key components of the uncertainty-aware CheXpert diagnosis system.

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import yaml

from uncertainty_aware_chexpert_diagnosis.data.loader import CheXpertDataModule
from uncertainty_aware_chexpert_diagnosis.models.model import EvidentialCheXpertModel
from uncertainty_aware_chexpert_diagnosis.evaluation.metrics import UncertaintyMetrics

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Configuration

In [ ]:
# Load config
config_path = Path.cwd().parent / 'configs' / 'default.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully")
print(f"Model architecture: {config['model']['architecture']}")
print(f"Number of classes: {config['model']['num_classes']}")
print(f"Uncertain policy: {config['data']['uncertain_policy']}")

## 2. Explore Data Distribution

Analyze the distribution of certain vs uncertain labels in CheXpert.

In [ ]:
# Initialize data module
data_module = CheXpertDataModule(config)
data_module.prepare_data()
data_module.setup()

print(f"Training samples: {len(data_module.train_dataset)}")
print(f"Validation samples: {len(data_module.val_dataset)}")

In [ ]:
# Analyze label distribution
train_labels = data_module.train_dataset.labels
train_uncertainty = data_module.train_dataset.uncertainty_masks

label_names = config['data']['labels']

# Compute statistics
positive_counts = (train_labels > 0.7).sum(axis=0)
negative_counts = (train_labels < 0.3).sum(axis=0)
uncertain_counts = train_uncertainty.sum(axis=0)

# Plot distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

x = np.arange(len(label_names))
width = 0.6

axes[0].bar(x, positive_counts, width, color='green', alpha=0.7)
axes[0].set_ylabel('Count')
axes[0].set_title('Positive Labels')
axes[0].set_xticks(x)
axes[0].set_xticklabels(label_names, rotation=45, ha='right')

axes[1].bar(x, negative_counts, width, color='red', alpha=0.7)
axes[1].set_ylabel('Count')
axes[1].set_title('Negative Labels')
axes[1].set_xticks(x)
axes[1].set_xticklabels(label_names, rotation=45, ha='right')

axes[2].bar(x, uncertain_counts, width, color='orange', alpha=0.7)
axes[2].set_ylabel('Count')
axes[2].set_title('Uncertain Labels')
axes[2].set_xticks(x)
axes[2].set_xticklabels(label_names, rotation=45, ha='right')

plt.tight_layout()
plt.show()

print(f"Total uncertain labels: {uncertain_counts.sum():.0f}")
print(f"Percentage uncertain: {100 * uncertain_counts.sum() / (len(train_labels) * len(label_names)):.2f}%")

## 3. Visualize Sample Images

In [ ]:
# Get a batch of samples
train_loader = data_module.train_dataloader()
batch = next(iter(train_loader))

images = batch['image']
labels = batch['label']
uncertainty_masks = batch['uncertainty_mask']

# Visualize first 4 samples
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx in range(4):
    img = images[idx].permute(1, 2, 0).numpy()
    # Denormalize
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    axes[idx].imshow(img)
    axes[idx].axis('off')
    
    # Add label information
    label_info = []
    for i, name in enumerate(label_names):
        if uncertainty_masks[idx, i] > 0.5:
            label_info.append(f"{name}: Uncertain")
        elif labels[idx, i] > 0.7:
            label_info.append(f"{name}: Positive")
        elif labels[idx, i] < 0.3:
            label_info.append(f"{name}: Negative")
    
    axes[idx].set_title('\n'.join(label_info[:3]), fontsize=8)

plt.tight_layout()
plt.show()

## 4. Model Architecture Exploration

In [ ]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = EvidentialCheXpertModel(config)
model = model.to(device)

total_params = model.get_num_params()
trainable_params = model.get_num_trainable_params()

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nModel architecture: {config['model']['architecture']}")
print(f"Use evidential: {config['model']['use_evidential']}")
print(f"Evidence activation: {config['model']['evidence_activation']}")

In [ ]:
# Test forward pass
model.eval()
with torch.no_grad():
    sample_images = images[:4].to(device)
    outputs = model(sample_images)

print("Model outputs:")
for key, value in outputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape {value.shape}")

## 5. Uncertainty Visualization

Visualize epistemic and aleatoric uncertainty for sample predictions.

In [ ]:
# Get predictions for a batch
model.eval()
with torch.no_grad():
    outputs = model(images.to(device))
    
    probs = outputs['prob'].cpu().numpy()
    epistemic = outputs['epistemic'].cpu().numpy() if 'epistemic' in outputs else None
    aleatoric = outputs['aleatoric'].cpu().numpy() if 'aleatoric' in outputs else None

if epistemic is not None and aleatoric is not None:
    # Plot uncertainty decomposition
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Epistemic uncertainty
    axes[0].hist(epistemic, bins=30, color='blue', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Epistemic Uncertainty')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Epistemic Uncertainty')
    axes[0].axvline(epistemic.mean(), color='red', linestyle='--', label=f'Mean: {epistemic.mean():.3f}')
    axes[0].legend()
    
    # Aleatoric uncertainty
    axes[1].hist(aleatoric, bins=30, color='green', alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Aleatoric Uncertainty')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of Aleatoric Uncertainty')
    axes[1].axvline(aleatoric.mean(), color='red', linestyle='--', label=f'Mean: {aleatoric.mean():.3f}')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(epistemic, aleatoric, alpha=0.6, c=probs.mean(axis=1), cmap='viridis')
    plt.colorbar(label='Mean Prediction Probability')
    plt.xlabel('Epistemic Uncertainty')
    plt.ylabel('Aleatoric Uncertainty')
    plt.title('Epistemic vs Aleatoric Uncertainty')
    plt.tight_layout()
    plt.show()
else:
    print("Uncertainty outputs not available (evidential mode disabled)")

## 6. Prediction Examples with Uncertainty

In [ ]:
# Show predictions with uncertainty for first 4 samples
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx in range(4):
    # Plot probabilities
    x = np.arange(len(label_names))
    axes[idx].bar(x, probs[idx], color='steelblue', alpha=0.7)
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(label_names, rotation=45, ha='right')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_ylim([0, 1])
    
    # Add uncertainty information if available
    if epistemic is not None:
        title = f"Sample {idx+1}\nEpistemic: {epistemic[idx]:.3f}, Aleatoric: {aleatoric[idx]:.3f}"
    else:
        title = f"Sample {idx+1}"
    
    axes[idx].set_title(title, fontsize=10)
    axes[idx].axhline(y=0.5, color='red', linestyle='--', alpha=0.3)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Innovation: Uncertainty Correlation

Demonstrate correlation between model uncertainty and radiologist uncertainty.

In [ ]:
if epistemic is not None:
    # Compute average uncertainty per sample
    radiologist_uncertainty = uncertainty_masks.numpy().mean(axis=1)
    
    # Scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(radiologist_uncertainty, epistemic, alpha=0.6, s=50)
    plt.xlabel('Radiologist Uncertainty (fraction of uncertain labels)')
    plt.ylabel('Model Epistemic Uncertainty')
    plt.title('Correlation between Radiologist and Model Uncertainty')
    
    # Add correlation coefficient
    from scipy.stats import spearmanr
    corr, p_value = spearmanr(radiologist_uncertainty, epistemic)
    plt.text(0.05, 0.95, f"Spearman ρ = {corr:.3f}\np-value = {p_value:.4f}",
             transform=plt.gca().transAxes, fontsize=12,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Add trend line
    z = np.polyfit(radiologist_uncertainty, epistemic, 1)
    p = np.poly1d(z)
    x_line = np.linspace(radiologist_uncertainty.min(), radiologist_uncertainty.max(), 100)
    plt.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
    
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nCorrelation Interpretation:")
    print(f"  Spearman correlation: {corr:.3f}")
    print(f"  This {'validates' if corr > 0.3 else 'suggests improvement needed for'} our approach:")
    print(f"  The model learns to be uncertain when radiologists are uncertain.")
else:
    print("Evidential mode is disabled. Enable it in config to see uncertainty correlation.")

## 8. Summary

This notebook demonstrated:
1. Data distribution with uncertain labels
2. Model architecture with evidential learning
3. Uncertainty decomposition (epistemic vs aleatoric)
4. Novel contribution: Learning to predict radiologist uncertainty

Next steps:
- Train the full model: `python scripts/train.py`
- Evaluate on test set: `python scripts/evaluate.py`
- Analyze ablation studies comparing different uncertainty policies